<a href="https://colab.research.google.com/github/amir-skh/fastbook/blob/master/cat-vs-dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # Choose kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amirskh","key":"2ca6efb96f725273e4039cfe91decbec"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!pip install kaggle

!kaggle competitions download -c dogs-vs-cats

!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip
!unzip -qq test1.zip

 99% 807M/812M [00:11<00:00, 127MB/s]
100% 812M/812M [00:11<00:00, 74.3MB/s]


In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
import os, shutil, pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [5]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False
)
conv_base.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [6]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [7]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 177ms/step - accuracy: 0.8098 - loss: 40.7681 - val_accuracy: 0.9550 - val_loss: 7.6063
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 139ms/step - accuracy: 0.9348 - loss: 9.2910 - val_accuracy: 0.9650 - val_loss: 5.4682
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 142ms/step - accuracy: 0.9640 - loss: 4.5510 - val_accuracy: 0.9750 - val_loss: 4.8390
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 141ms/step - accuracy: 0.9611 - loss: 4.0121 - val_accuracy: 0.9730 - val_loss: 2.7858
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 175ms/step - accuracy: 0.9574 - loss: 4.5506 - val_accuracy: 0.9760 - val_loss: 4.0059
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 175ms/step - accuracy: 0.9665 - loss: 3.8982 - val_accuracy: 0.9760 - val_loss: 4.5910
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 140ms/step - accuracy: 0.9671 - loss: 4.2698 - val_accuracy: 0.9630 - val_loss: 6.7977
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 142ms/step - accuracy: 0.9734 - loss: 2.8994 - val_accura

In [9]:
test_model = keras.models.load_model(
"feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - accuracy: 0.9806 - loss: 1.6133
Test accuracy: 0.983


In [10]:
from google.colab import files
files.download("feature_extraction_with_data_augmentation.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
%%writefile app.py
import gradio as gr
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import img_to_array

# Load your trained model
model = keras.models.load_model("feature_extraction_with_data_augmentation.keras")

# Match the same image size as training
IMG_SIZE = (180, 180)

def predict(image):
    image = image.convert("RGB").resize(IMG_SIZE)
    arr = img_to_array(image)
    arr = np.expand_dims(arr, axis=0)
    arr = keras.applications.vgg16.preprocess_input(arr)

    prob = model.predict(arr, verbose=0)[0][0]
    label = "Dog" if prob >= 0.5 else "Cat"
    conf = prob if prob >= 0.5 else 1 - prob
    return {"Prediction": label, "Confidence": float(conf)}

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload a cat or dog photo"),
    outputs="json",
    title="Cat vs Dog Classifier",
    description="Upload an image, and the model will tell you if it's a cat or dog."
)

if __name__ == "__main__":
    demo.launch(share=True)

Overwriting app.py


In [15]:
!pip install gradio

In [16]:
!python app.py

2025-09-01 22:17:56.211406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756765076.232059    9363 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756765076.238298    9363 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756765076.254294    9363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756765076.254325    9363 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756765076.254331    9363 computation_placer.cc:177] computation placer alr

In [17]:
from google.colab import files
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
%%writefile requirements.txt
gradio
tensorflow

Writing requirements.txt


In [19]:
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>